In [79]:
# Now we are importing some important modules
#json- for loading json files to memory

import json
from time import time
from stringMatcher import levenshtein_ratio_and_distance

# pretty table for showing output in tabular form

from prettytable import PrettyTable
table = PrettyTable(field_names=["Item_Type","Order_ID","Price"])


# Load the database into the memory

In [80]:
data = json.load(open("newdataFinal_GIS.json.", "rb"))

# Load the Global secondary index into memory

In [81]:
GIS = json.load(open("Order_cost1.json", "rb"))

# Write a function to search the order details by orderid

In [90]:
def getClosestMatch(queryString):
    
    # Initialize the time counter...
    tic                  = time()
    
    # Find closest match of the queryString from the Global Secondary Index...
    matchRatios          = [levenshtein_ratio_and_distance(queryString, KEY) for KEY, _ in GIS.items()]
    bestMatchRatio       = max(matchRatios)
    

    # If the best match ratio is less than 50% then we will assume that the records doesn't exist...
    if (bestMatchRatio < 0.5):
        return {
"response code" : 404,
            "search time"   : f"{time()-tic} seconds",
            "message"       : "error",
            "response body" : "no match found"
        }
    
    # Otherwise...
    bestMatchRatio_index = matchRatios.index(bestMatchRatio)

    # Find the Order_ID of the order which is the best match to the queryString...
    bestMatch_index = list(GIS)[bestMatchRatio_index]
    bestMatch_Order_ID= GIS[bestMatch_index]
    
        
    orderInfo= data[str(bestMatch_Order_ID)]

    
    
    # Mark the search completion time...
    toc                  = time()

    # Add order id to the response...
    orderInfo["Order_ID"] = bestMatch_Order_ID
    # Construct the response Model...
    responseModel = {
        "response code" : 200,
        "search time"   : f"{toc-tic} seconds",
        "message"       : "success",
        "response body" : orderInfo
    }
    
    return responseModel

#  Write a function to make a request and visualize the data returned by the search function

In [91]:
def request(orderToSearch=""):
    # order id should always be in a string...
    orderToSearch = str(orderToSearch)
    # Check if the order id is valid...
    if(orderToSearch.replace(" ", "").strip() == ""):
        return f"Invalid OrderID"
    
    # Make a request to fetch the order info...
    response = getClosestMatch(queryString = orderToSearch)

    # Check the response code...
    if response["response code"] == 404:
        return f"ERROR: No records found"
    
    
    Order_ID         : {response['response body']['Order ID']}
    
    Order_date          : {response['response body']['Order Date']}
    Ship_date         :{response['response body']['Ship Date']}
    Price             :{response['response body']['Price']}
    print(table)
    
   

### finally make a request.

In [92]:
request(orderToSearch="1")

'ERROR: No records found'

In [93]:
request(orderToSearch="102928006")

KeyError: "{'Country': 'India', 'Itemtype': 'Personal Care', 'Order_ID': 102928006, 'Orderdate': '12/24/2015', 'Region': 'Asia', 'Shipdate': '1/31/2016', 'costs': {'Number': 7539, 'Price': 81.73, 'Totalcost': 427235.13}}"